In [4]:
import requests
#for HTML manipulation and pulling option chain
from bs4 import BeautifulSoup,SoupStrainer
import traceback
import pandas as pd
import calendar
import datetime
import time
import os
import csv
import re
import urllib.parse
import urllib
from urllib.request import Request,urlopen
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.action_chains import ActionChains
import winsound
from fake_useragent import UserAgent

In [2]:
ua = UserAgent()
username = os.getenv("USERNAME")
binary = FirefoxBinary(r'C:\Program Files\Mozilla Firefox\firefox.exe')
fp = (r'C:\Users\Yash Gupta\AppData\Roaming\Mozilla\Firefox\Profiles\q7imc2hk.yashscrape')
opts = Options()
opts.profile = fp
firefox_capabilities = DesiredCapabilities.FIREFOX
firefox_capabilities["pageLoadStrategy"] = "normal"
opts.add_argument(f'user-agent=ua.chrome')
link = 'https://marketplace.qantas.com/au/brands'
headers = {
        "User-Agent": ua.random,
    }

seller_links = []
product_links = []
seller_detail = []
error_list = []

driver = webdriver.Firefox(capabilities=firefox_capabilities,firefox_binary=binary, options = opts)
driver.get(link)
soup = BeautifulSoup(driver.page_source, "html.parser")

C:\Users\Yash Gupta\AppData\Local\Temp\ipykernel_17428\1001394963.py:6: DeprecationWarning: Setting a profile has been deprecated. Please use the set_preference and install_addons methods
  opts.profile = fp
C:\Users\Yash Gupta\AppData\Local\Temp\ipykernel_17428\1001394963.py:20: DeprecationWarning: capabilities and desired_capabilities have been deprecated, please pass in a Service object
  driver = webdriver.Firefox(capabilities=firefox_capabilities,firefox_binary=binary, options = opts)
C:\Users\Yash Gupta\AppData\Local\Temp\ipykernel_17428\1001394963.py:20: DeprecationWarning: firefox_binary has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(capabilities=firefox_capabilities,firefox_binary=binary, options = opts)


In [3]:
# Get seller links
#CLass names change everytime so manual updated will be required 
for block in soup.find("div",{"class":"sc-WsMwQ fazlPT"}):
    for block2 in block.find_all("ul",{"class":"sc-kUdmhA kWuKnC"}):
        for li in block2.find_all("li",{"class":"sc-hgRRfv htwhCt"}):
            seller_links.append( {'seller_name': li.text,'seller_link':li.find("a")['href'] })
df_seller_links = pd.DataFrame(seller_links)
df_seller_links["seller_link"] =  "https://marketplace.qantas.com"+df_seller_links["seller_link"] 
df_seller_links

,seller_name,seller_link
0,100%,https://marketplace.qantas.com/au/c/100
1,2XU,https://marketplace.qantas.com/au/c/2-xu
2,360 Degrees,https://marketplace.qantas.com/au/c/360-degrees
3,4M,https://marketplace.qantas.com/au/c/4-m
4,4Moms,https://marketplace.qantas.com/au/c/4-moms
...,...,...
1052,Ziegler & Brown,https://marketplace.qantas.com/au/c/ziegler-brown
1053,Ziggy by Ziegler and Brown,https://marketplace.qantas.com/au/c/ziggy-by-z...
1054,ZLOS,https://marketplace.qantas.com/au/c/zlos
1055,Zwilling,https://marketplace.qantas.com/au/c/zwilling


In [87]:
#rebuild code to get product links
#THIS WONT WORK - CLICK THROUGH PAGES 
product_links = []
for link in df_seller_links['seller_link'].tolist():
    next_page_link = link
    #Check if first image is loaded
    driver.get(link)
    driver.maximize_window()
    while next_page_link != None:
        try:
            time.sleep(5)
            WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[3]/div[3]/div/div[4]/div[2]/div[1]/div[1]/div[1]/div[1]/a/img')))
            block = driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/div[3]/div/div[4]/div[2]/div[1]')
            for e in block.find_elements(By.TAG_NAME, "a"):
                product_links.append({'seller_link':link ,'product_link':e.get_attribute("href")} )
        except:
            print('No Products Found: ', link)
        try:
            next_page = driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/div[3]/div/div[4]/div[2]/div[2]/div[2]')
            next_page_link = next_page.find_element(By.XPATH,'//a[@data-testid = "page-next"]')
            next_page_link.click()
        except:
            next_page_link=None
            break
df_product_links = pd.DataFrame(product_links)
df_product_links

No Products Found:  https://marketplace.qantas.com/au/c/4-paws
No Products Found:  https://marketplace.qantas.com/au/c/alex-and-trahanas
No Products Found:  https://marketplace.qantas.com/au/c/b-w
No Products Found:  https://marketplace.qantas.com/au/c/ben-q
No Products Found:  https://marketplace.qantas.com/au/c/better-homes-gardens
No Products Found:  https://marketplace.qantas.com/au/c/climate-positive-delivery
No Products Found:  https://marketplace.qantas.com/au/c/deluxe-materials
No Products Found:  https://marketplace.qantas.com/au/c/eight-bob
No Products Found:  https://marketplace.qantas.com/au/c/engel
No Products Found:  https://marketplace.qantas.com/au/c/f-falcon
No Products Found:  https://marketplace.qantas.com/au/c/fathers-day
No Products Found:  https://marketplace.qantas.com/au/c/fit-smart
No Products Found:  https://marketplace.qantas.com/au/c/food-saver
No Products Found:  https://marketplace.qantas.com/au/c/happy-hopperz
No Products Found:  https://marketplace.qanta

In [8]:
#based on product links get relevent details
product_detail = []
link = 'https://marketplace.qantas.com/au/p/country-road-australian-cotton-printed-heritage-shopper/AU100191558'
# for link in df_product_links['product_link']:
#     if link not in df_temp['product_link'].tolist():
#/html/body/div[2]/div[3]/div[1]/div[1]
hierarchy = ''
driver.get(link)
driver.maximize_window()
time.sleep(1)
try:
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[3]/div[1]/div[1]')))
    hierarchy_element = driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/div[1]/div[1]')
    for e in hierarchy_element.find_elements(By.TAG_NAME, 'a'):
        hierarchy = hierarchy+ '/' + e.get_attribute("innerHTML") 
except:
    hierarchy = 'N/A'
    print("Hierarchy Not Found:", link)
time.sleep(2)



body1 = []
body_list = []
try:  
    body1_element =  driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/div[1]/div[3]/div[2]/div[1]')
    #print(body1_element.get_attribute("innerHTML"))
    i=0
    for e in body1_element.find_elements(By.TAG_NAME, 'div'):
        if((i>=1) and (i<=4)):
            if ('Review' in e.text):
                i = i-1
#                continue
            body1.append(e.text)
        i=i+1

    body_list.append(body1[0])
    body_list.append(re.search(r"SKU:\s*(\w+)", body1[1]).group(1) if re.search(r"SKU:\s*(\w+)", body1[1]) else None)
    body_list.append(body1[2].replace('Sold and delivered by ', ''))
    # body_list.append(re.findall(r'\d+(?:,\d+)*(?:\.\d+)?', body1[3])[0])
    # body_list.append(re.findall(r'\d+(?:,\d+)*(?:\.\d+)?', body1[3])[1])
    body_list.append(body1[3].replace('\n', ' '))
except:
    print("BOdy 1 not found", link)
    body_list.append('N/A')
    body_list.append('N/A')
    body_list.append('N/A')
    body_list.append('N/A')
try:
    body2_elements = driver.find_elements(By.XPATH,'/html/body/div[1]/div[3]/div[1]/div[3]/div[2]/div[*]/div[2]')
    for e in body2_elements:
        if ('Dispatch' in e.text):
            body2_element = e.text
#            break
    body_list.append(body2_element.split('\n')[0])
    body3_elements = driver.find_elements(By.XPATH,'/html/body/div[1]/div[3]/div[1]/div[3]/div[2]/div[*]/div[1]')
    for e in body3_elements:
        if (' delivery' in e.text):
            body3_element = e.text
#            break
    body_list.append(body3_element.split('\n')[0])
    body_list.append(body3_element.split('\n')[1])
except:
    print('Delivery not found: ',link)
    body_list.append('N/A')
    body_list.append('N/A')
    body_list.append('N/A')
try:
    product_detail.append( {'brand': body_list[0],
                        'product_name': hierarchy.rsplit('/', 1)[1],
                        'sku':body_list[1],
                        'sold_delivered_by':body_list[2],
                        'price':body_list[3],
                        'dispatch':body_list[4],
                        'delivery_type':body_list[5],
                        'delivery_cost':body_list[6],
                        'hierarchy': hierarchy.rsplit('/', 1)[0],
                        'product_link':link
                       })
except:
    print('No data')
#    continue
print(product_detail)
df_product_detail= pd.DataFrame(product_detail)    
df_product_detail
with open('product_detail_live.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow({'brand': body_list[0],
                        'product_name': 'hierarchy1',
                        'sku':body_list[1],
                        'sold_delivered_by':body_list[2],
                        'price':body_list[3],
                        'dispatch':body_list[4],
                        'delivery_type':body_list[5],
                        'delivery_cost':body_list[6],
                        'hierarchy': 'hierarchy2',
                        'product_link':'response.request.url'
                       }) # Use writerow for single list

Delivery not found:  https://marketplace.qantas.com/au/p/country-road-australian-cotton-printed-heritage-shopper/AU100191558
No data
[]


In [13]:
with open('product_detail_live.csv', "a") as csvfile:
                headers = ['brand', 'product_name','sku','sold_delivered_by','price','dispatch','delivery_type','delivery_cost','hierarchy','product_link']
                writer = csv.DictWriter(csvfile, delimiter=',', lineterminator='\n',fieldnames=headers)
                if os.stat('product_detail_live.csv').st_size == 0:
                        writer.writeheader()  # file doesn't exist yet, write a header
                writer.writerow({'brand': body_list[0],
                                'product_name': 'hierarchy1',
                                'sku':body_list[1],
                                'sold_delivered_by':body_list[2],
                                'price':body_list[3],
                                'dispatch':body_list[4],
                                'delivery_type':body_list[5],
                                'delivery_cost':body_list[6],
                                'hierarchy': 'hierarchy2',
                                'product_link':'response.request.url'
                               })

In [152]:

df_product_detail= pd.DataFrame(product_detail)    
df_product_detail
df_temp = df_product_detail.copy()

In [153]:
df_temp

,brand,product_name,sku,sold_delivered_by,price,dispatch,delivery_type,delivery_cost,hierarchy,product_link
0,100%,Speedcraft XS - Soft Tact Black - Smoke,AU59664,Qantas Marketplace,"$229.00 or 39,830 PTS",Dispatches in 5 days,Standard delivery,"1,200 PTS or $8.00",/Home/Womens Fashion/Accessories/Sunglasses/Po...,https://marketplace.qantas.com/au/p/100percent...
1,100%,Speedcraft XS - Soft Tact Black - HiPER Red,AU59663,Qantas Marketplace,"$279.00 or 48,530 PTS",Dispatches in 5 days,Standard delivery,"1,200 PTS or $8.00",/Home/Womens Fashion/Accessories/Sunglasses/Po...,https://marketplace.qantas.com/au/p/100percent...
2,2XU,Light Speed Mid-Rise Compression Tight (WA6523b),AU15449,Qantas Marketplace,"$169.00 or 29,400 PTS",Dispatches in 3 days,Standard delivery,"1,200 PTS or $8.00",/Home/Womens Fashion/Clothing/Womens Activewea...,https://marketplace.qantas.com/au/p/2xu-light-...
3,2XU,Light Speed Compression Tights (MA5305b),AU15646,Qantas Marketplace,"$169.00 or 29,400 PTS",Dispatches in 3 days,Standard delivery,"1,200 PTS or $8.00",/Home/Mens Fashion/Clothing/Sportswear,https://marketplace.qantas.com/au/p/2xu-light-...
4,2XU,Light Speed Compression Tights (MA5305b),AU15680,Qantas Marketplace,"$169.00 or 29,400 PTS",Dispatches in 3 days,Standard delivery,"1,200 PTS or $8.00",/Home/Mens Fashion/Clothing/Sportswear,https://marketplace.qantas.com/au/p/2xu-light-...
...,...,...,...,...,...,...,...,...,...,...
4985,Beurer,LR405 Bluetooth Triple Filter Air Purifier: 69m2,AU217927,Qantas Marketplace,"$499.00 or 86,790 PTS",Dispatches in 3 days,Standard delivery,"3,000 PTS or $20.00",/Home/Appliances/Heating Cooling Air Treatment...,https://marketplace.qantas.com/au/p/beurer-lr4...
4986,Beurer,BM27LE Upper Arm Blood Pressure Monitor: Limit...,AU214170,Qantas Marketplace,"$100.00 or 17,400 PTS",Dispatches in 3 days,Standard delivery,"1,200 PTS or $8.00",/Home/Appliances/Health Personal Care/Health B...,https://marketplace.qantas.com/au/p/beurer-bm2...
4987,Beurer,HC45OCEAN 2-in-1 Volumising Hair Dryer Brush,AU189488,Qantas Marketplace,"$69.00 or 12,000 PTS",Dispatches in 2 days,Standard delivery,"2,250 PTS or $15.00",/Home/Appliances/Hair Styling/Hair Dryers,https://marketplace.qantas.com/au/p/beurer-hc4...
4988,Beurer,BC54 Bluetooth Wrist Blood Pressure Monitor,AU211355,Qantas Marketplace,"$100.00 or 17,400 PTS",Dispatches in 3 days,Standard delivery,"2,250 PTS or $15.00",/Home/Appliances/Health Personal Care/Health B...,https://marketplace.qantas.com/au/p/beurer-bc5...


In [141]:
#df_product_links[10000:20000].to_csv('product_links_10k.csv')

In [149]:
df_seller_links.to_csv('qantas_product_detail_final.csv')

In [151]:
df_product_links.to_csv('qantas_product_links.csv')

In [154]:
df_product_detail.to_csv('qantas_product_detail1.csv')

In [46]:
df_t = pd.read_csv('qantas_product_detail_final.csv')

In [47]:
df_t2 = pd.read_csv('qantas_product_detail3.csv')

In [48]:
df_t3 = pd.concat([df_t,df_t2])

In [49]:
df_t3.drop(columns=df_t3.columns[0], axis=1,  inplace=True)

In [50]:
df_t3.drop_duplicates(inplace = True)

In [52]:
df_t3.to_csv('qantas_product_detail_final.csv')

In [51]:
df_t3

,brand,product_name,sku,sold_delivered_by,price,dispatch,delivery_type,delivery_cost,hierarchy,product_link
0,100%,Speedcraft XS - Soft Tact Black - Smoke,AU59664,Qantas Marketplace,"$229.00 or 39,830 PTS",Dispatches in 5 days,Standard delivery,"1,200 PTS or $8.00",/Home/Womens Fashion/Accessories/Sunglasses/Po...,https://marketplace.qantas.com/au/p/100percent...
1,100%,Speedcraft XS - Soft Tact Black - HiPER Red,AU59663,Qantas Marketplace,"$279.00 or 48,530 PTS",Dispatches in 5 days,Standard delivery,"1,200 PTS or $8.00",/Home/Womens Fashion/Accessories/Sunglasses/Po...,https://marketplace.qantas.com/au/p/100percent...
2,2XU,Light Speed Mid-Rise Compression Tight (WA6523b),AU15449,Qantas Marketplace,"$169.00 or 29,400 PTS",Dispatches in 3 days,Standard delivery,"1,200 PTS or $8.00",/Home/Womens Fashion/Clothing/Womens Activewea...,https://marketplace.qantas.com/au/p/2xu-light-...
3,2XU,Light Speed Compression Tights (MA5305b),AU15646,Qantas Marketplace,"$169.00 or 29,400 PTS",Dispatches in 3 days,Standard delivery,"1,200 PTS or $8.00",/Home/Mens Fashion/Clothing/Sportswear,https://marketplace.qantas.com/au/p/2xu-light-...
4,2XU,Light Speed Compression Tights (MA5305b),AU15680,Qantas Marketplace,"$169.00 or 29,400 PTS",Dispatches in 3 days,Standard delivery,"1,200 PTS or $8.00",/Home/Mens Fashion/Clothing/Sportswear,https://marketplace.qantas.com/au/p/2xu-light-...
...,...,...,...,...,...,...,...,...,...,...
131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://marketplace.qantas.com/au/p/tutu-du-mo...
132,Trudon,Giambattista Valli Rose Poivree Candle,AU56931,Qantas Marketplace,"$149.00 or 25,920 PTS",Dispatches in 3 days,Standard delivery,"2,250 PTS or $15.00",/Home/Home/Home Decor/Scented Candles &amp; Di...,https://marketplace.qantas.com/au/p/trudon-gia...
133,Tutu Du Monde,Gemini Fringe Shawl Linen,AU529648,Qantas Marketplace,"$163.00 or 28,350 PTS",Dispatches in 3 days,Standard delivery,"1,800 PTS or $12.00",/Home/Baby Kids Toys/Clothing Footwear Accesso...,https://marketplace.qantas.com/au/p/tutu-du-mo...
134,Trudon,Abd el Kader Room Spray,AU56721,Qantas Marketplace,"$299.00 or 52,000 PTS",Dispatches in 3 days,Standard delivery,"2,250 PTS or $15.00",/Home/Home/Home Decor/Scented Candles &amp; Di...,https://marketplace.qantas.com/au/p/trudon-abd...
